In [19]:
import json
import random
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# JSON 파일 읽기
with open('./noun_frequencies.json', 'r', encoding='utf-8') as file:
    noun_frequencies = json.load(file)

# 5개의 단어 랜덤 추출
random_nouns = random.sample(list(noun_frequencies.keys()), 5)
print("랜덤 추출된 단어:", random_nouns)

# 크리에이터 데이터프레임 읽기
df = pd.read_csv('./DataFrame2.csv')

# '아이템명' 열과 랜덤 추출된 단어들의 유사도 계산
new_df = df[['크리에이터명', '아이템명']]
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(new_df['아이템명'].tolist() + random_nouns)
cosine_sim = cosine_similarity(tfidf_matrix[-len(random_nouns):], tfidf_matrix[:-len(random_nouns)])

# 유사도가 높은 10개의 크리에이터 추출
similarity_scores = cosine_sim.mean(axis=0)
top_indices = similarity_scores.argsort()[-10:][::-1]
top_creators = new_df.iloc[top_indices]

# 유사도 점수를 퍼센트로 변환하여 데이터프레임에 추가
top_creators['유사도'] = similarity_scores[top_indices] * 100

print("유사도가 높은 10개의 크리에이터:")
print(top_creators)
top_creators.head()

랜덤 추출된 단어: ['최초', '향촌', '전국', '온오프라인', '매입한']
유사도가 높은 10개의 크리에이터:
         크리에이터명                                               아이템명       유사도
6      주식회사 지웍스  ▶ “대구 팔공산 왕건 사과빵 개발” - 100년된 대구 팔공산 왕건길 이야기와 평...  4.085760
10      (주)메디프랜  ▶ “대구 반야월지역 연근 추출물을 사용한 메디시티 대구 천연치약 제조 판매” - ...  3.396498
3          와룡총각  ▶ “한돈 능이버섯 수제 떡갈비” - 대구 와룡시장의 식자재를 활용한 다양한 종류의...  3.321757
7            지야  ▶ “판매되지 못한 대구 로컬 섬유 제품을 활용한 업사이클링 데스크테리어 제품 개발...  3.289429
5   주식회사 먼데인먼데이  ▶ “빅 데이터 기반 현대인 발 형태에 최적화된 신규 사이즈 체계 적용 맞춤 구두”...  2.821162
24         루프세터  ▶ 대구 최초 백화점 ‘무영당’의 역사를 계승하는 ‘인터렉티브 미디어 아트’ 콘텐츠...  2.801497
33        디스모먼트  로컬 폐기솜을 활용한 다견가정, 중대형견을 위한 전용 카시트 - 전국 침구(이불)제...  2.755146
14        그린페이스  ▶ “우범 지대였던 대구 두류공원을 동화속 소풍을 컨셉으로 체험서비스제공” - 대구...  0.000000
13     가게(GAGE)  ▶ “대구 안경산업 인프라를 활용하여 제조한 멀티안구 적용가능한 초기능성 안전 안경...  0.000000
12   cmnmstudio  ▶ “대구 원단업체와 대구 디자이너를 연결하여 대구패션 협업 생태계 구축” - 대구...  0.000000


/var/folders/5c/_d3k15dx5sx7j_rn5lcmwyqr0000gn/T/ipykernel_83339/2615529965.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_creators['유사도'] = similarity_scores[top_indices] * 100


,크리에이터명,아이템명,유사도
6,주식회사 지웍스,▶ “대구 팔공산 왕건 사과빵 개발” - 100년된 대구 팔공산 왕건길 이야기와 평...,4.085760
10,(주)메디프랜,▶ “대구 반야월지역 연근 추출물을 사용한 메디시티 대구 천연치약 제조 판매” - ...,3.396498
3,와룡총각,▶ “한돈 능이버섯 수제 떡갈비” - 대구 와룡시장의 식자재를 활용한 다양한 종류의...,3.321757
7,지야,▶ “판매되지 못한 대구 로컬 섬유 제품을 활용한 업사이클링 데스크테리어 제품 개발...,3.289429
5,주식회사 먼데인먼데이,▶ “빅 데이터 기반 현대인 발 형태에 최적화된 신규 사이즈 체계 적용 맞춤 구두”...,2.821162


In [20]:
import json
import random
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# JSON 파일 읽기
with open('./noun_frequencies.json', 'r', encoding='utf-8') as file:
    noun_frequencies = json.load(file)

# 5개의 단어 랜덤 추출
random_nouns = random.sample(list(noun_frequencies.keys()), 5)
print("랜덤 추출된 단어:", random_nouns)

# 크리에이터 데이터프레임 읽기
df = pd.read_csv('./DataFrame2.csv')

# '아이템명' 열과 랜덤 추출된 단어들의 유사도 계산
new_df = df[['크리에이터명', '아이템명']]

# SBERT 모델 로드
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# 문장 임베딩 생성
item_embeddings = model.encode(new_df['아이템명'].tolist(), convert_to_tensor=True)
noun_embeddings = model.encode(random_nouns, convert_to_tensor=True)

# 유사도 계산
cosine_sim = util.pytorch_cos_sim(noun_embeddings, item_embeddings)

# 유사도가 높은 10개의 크리에이터 추출
similarity_scores = cosine_sim.mean(dim=0).cpu().numpy()
top_indices = similarity_scores.argsort()[-10:][::-1]
top_creators = new_df.iloc[top_indices]

# 유사도 점수를 퍼센트로 변환하여 데이터프레임에 추가
top_creators['유사도'] = similarity_scores[top_indices] * 100

print("유사도가 높은 10개의 크리에이터:")
print(top_creators)
top_creators.head()

/opt/homebrew/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


랜덤 추출된 단어: ['식사대용', '대구시', '판매', '영릉향', '방석']


/opt/homebrew/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


유사도가 높은 10개의 크리에이터:
         크리에이터명                                               아이템명        유사도
22    주식회사 케이센스  ▶ “대구 약령시 한약재를 활용한 힐링트렌드 한방향제(韓芳香劑)개발” - 동의보감 ...  63.902081
33        디스모먼트  로컬 폐기솜을 활용한 다견가정, 중대형견을 위한 전용 카시트 - 전국 침구(이불)제...  62.643116
1           애니먼  ▶ “한약재를 베이스로 한 반려동물 기능성 식품” - 대구 약령시의 양질의 한약 재...  62.048923
31     칠성반고개무침회  ▶ “대구 10味 반고개 무침회 밀키트” - 대구 10味로 사랑받고 있는 반고개 무...  61.747108
6      주식회사 지웍스  ▶ “대구 팔공산 왕건 사과빵 개발” - 100년된 대구 팔공산 왕건길 이야기와 평...  61.246395
17       북성로공구빵  ▶ “대구 최초의 근대식 공단이 위치했던 북성로 공구거리를 상징하는 공구모양의 빵 ...  60.349537
5   주식회사 먼데인먼데이  ▶ “빅 데이터 기반 현대인 발 형태에 최적화된 신규 사이즈 체계 적용 맞춤 구두”...  60.299652
23          플라낭  ▶ “대구불로 화훼 농장을 활용한 직거래 온라인 플라워샵” - 대구불로 화훼농원의 ...  60.190193
10      (주)메디프랜  ▶ “대구 반야월지역 연근 추출물을 사용한 메디시티 대구 천연치약 제조 판매” - ...  60.069771
3          와룡총각  ▶ “한돈 능이버섯 수제 떡갈비” - 대구 와룡시장의 식자재를 활용한 다양한 종류의...  60.044693


/var/folders/5c/_d3k15dx5sx7j_rn5lcmwyqr0000gn/T/ipykernel_83339/3836216492.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_creators['유사도'] = similarity_scores[top_indices] * 100


,크리에이터명,아이템명,유사도
22,주식회사 케이센스,▶ “대구 약령시 한약재를 활용한 힐링트렌드 한방향제(韓芳香劑)개발” - 동의보감 ...,63.902081
33,디스모먼트,"로컬 폐기솜을 활용한 다견가정, 중대형견을 위한 전용 카시트 - 전국 침구(이불)제...",62.643116
1,애니먼,▶ “한약재를 베이스로 한 반려동물 기능성 식품” - 대구 약령시의 양질의 한약 재...,62.048923
31,칠성반고개무침회,▶ “대구 10味 반고개 무침회 밀키트” - 대구 10味로 사랑받고 있는 반고개 무...,61.747108
6,주식회사 지웍스,▶ “대구 팔공산 왕건 사과빵 개발” - 100년된 대구 팔공산 왕건길 이야기와 평...,61.246395


In [29]:
import json
import random
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import numpy as np

# JSON 파일 읽기
with open('./noun_frequencies.json', 'r', encoding='utf-8') as file:
    noun_frequencies = json.load(file)

# 5개의 단어 랜덤 추출
random_nouns = random.sample(list(noun_frequencies.keys()), 5)
print("랜덤 추출된 단어:", random_nouns)

# 크리에이터 데이터프레임 읽기
df = pd.read_csv('./DataFrame2.csv')

# '아이템명' 열과 랜덤 추출된 단어들의 유사도 계산
new_df = df[['크리에이터명', '아이템명']]

# 최신 SBERT 모델 로드
model = SentenceTransformer('all-mpnet-base-v2')

# 문장 임베딩 생성
item_embeddings = model.encode(new_df['아이템명'].tolist(), convert_to_tensor=True)
noun_embeddings = model.encode(random_nouns, convert_to_tensor=True)

# 유사도 계산
cosine_sim = util.pytorch_cos_sim(noun_embeddings, item_embeddings)

# 유사도가 높은 10개의 크리에이터 추출
similarity_scores = cosine_sim.mean(dim=0).cpu().numpy()
top_indices = similarity_scores.argsort()[-10:][::-1]
top_creators = new_df.iloc[top_indices]

# 유사도 점수를 퍼센트로 변환하여 데이터프레임에 추가
top_creators['유사도'] = similarity_scores[top_indices] * 100

print("유사도가 높은 10개의 크리에이터:")
print(top_creators)
top_creators.head()

랜덤 추출된 단어: ['비효율적인', '여행할', '현대', '수요망', '전통의류']


/opt/homebrew/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


유사도가 높은 10개의 크리에이터:
       크리에이터명                                               아이템명        유사도
4         우니스  ▶ “대구안경산업공단 구성원들의 시간과 공간을 담은 웹 드로잉 및 애니메이션” - ...  62.786247
21  주식회사 아트라미  ▶ “독립 아티스트와 지역을 연결하는 복합문화공간” - 대구지역의 대구예술발전소, ...  59.780949
8          차림  ▶ “대구 지역 자연환경 및 지역기반제조 기술을 활용한 생활한복 제작” - 원단구매...  59.479584
32   주식회사 아트숨  대구 지역 기반 뮤지션 및 대명 공연거리 - 대구 지역의 뮤지션 및 대명 공연거리의...  59.212803
28       제이씨디  ▶ 달서구 문화재, 선사시대로 활성화 관광 플랫폼 - 달서구 문화재, 선사시대로 활...  59.190033
14      그린페이스  ▶ “우범 지대였던 대구 두류공원을 동화속 소풍을 컨셉으로 체험서비스제공” - 대구...  58.840076
13   가게(GAGE)  ▶ “대구 안경산업 인프라를 활용하여 제조한 멀티안구 적용가능한 초기능성 안전 안경...  58.670521
34      모던글로벌  대구-경북의 참외, 홍삼, 한약재를 이용하여 면역력을 강화한 식사대용 다이어트 분말...  58.571251
23        플라낭  ▶ “대구불로 화훼 농장을 활용한 직거래 온라인 플라워샵” - 대구불로 화훼농원의 ...  58.261120
1         애니먼  ▶ “한약재를 베이스로 한 반려동물 기능성 식품” - 대구 약령시의 양질의 한약 재...  58.177948


/var/folders/5c/_d3k15dx5sx7j_rn5lcmwyqr0000gn/T/ipykernel_83339/2078717790.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_creators['유사도'] = similarity_scores[top_indices] * 100


,크리에이터명,아이템명,유사도
4,우니스,▶ “대구안경산업공단 구성원들의 시간과 공간을 담은 웹 드로잉 및 애니메이션” - ...,62.786247
21,주식회사 아트라미,"▶ “독립 아티스트와 지역을 연결하는 복합문화공간” - 대구지역의 대구예술발전소, ...",59.780949
8,차림,▶ “대구 지역 자연환경 및 지역기반제조 기술을 활용한 생활한복 제작” - 원단구매...,59.479584
32,주식회사 아트숨,대구 지역 기반 뮤지션 및 대명 공연거리 - 대구 지역의 뮤지션 및 대명 공연거리의...,59.212803
28,제이씨디,"▶ 달서구 문화재, 선사시대로 활성화 관광 플랫폼 - 달서구 문화재, 선사시대로 활...",59.190033
